In [2]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta
from google.cloud import bigquery
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from pycaret.regression import *

In [3]:
# 📌 Set Up BigQuery Connection
PROJECT_ID = "travel-insider-452211"
DATASET_NAME = "travel_insider_dataset"
TABLE_NAME = "filtered_flights"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/sebastian/code/JPYY-96/travel_insider/raw_data/travel-insider-452211-181bd2eba48e.json"

# 📌 Initialize BigQuery Client
client = bigquery.Client()
query = f"SELECT * FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}` ORDER BY RAND()  LIMIT 100000"

# 📌 Load Data in Chunks with Timer
chunk_size = 1000000
data_chunks = []
start_time = time.time()

print(f"⏳ [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loading data from BigQuery in chunks...")
query_job = client.query(query)
result_iter = query_job.result(page_size=chunk_size)

for page in result_iter.pages:
    chunk_data = [dict(row) for row in page]
    df_chunk = pd.DataFrame(chunk_data)
    data_chunks.append(df_chunk)
    print(f"✅ [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loaded {len(df_chunk)} rows (total: {sum(len(c) for c in data_chunks)})")

data_query = pd.concat(data_chunks, ignore_index=True)
print(f"✅ [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Successfully loaded {len(data_query)} rows from BigQuery!")


⏳ [2025-03-13 20:15:44] Loading data from BigQuery in chunks...
✅ [2025-03-13 20:15:56] Loaded 31818 rows (total: 31818)
✅ [2025-03-13 20:16:03] Loaded 31818 rows (total: 63636)
✅ [2025-03-13 20:16:12] Loaded 31818 rows (total: 95454)
✅ [2025-03-13 20:16:13] Loaded 4546 rows (total: 100000)
✅ [2025-03-13 20:16:13] Successfully loaded 100000 rows from BigQuery!


In [4]:
# 📌 Convert Date Columns
data_query["searchDate"] = pd.to_datetime(data_query["searchDate"])
data_query["flightDate"] = pd.to_datetime(data_query["flightDate"])

# 📌 Feature Engineering
data_query["days_to_flight"] = (data_query["flightDate"] - data_query["searchDate"]).dt.days
data_query["day_of_week"] = data_query["flightDate"].dt.dayofweek
data_query["is_weekend"] = (data_query["day_of_week"] >= 5).astype(int)
data_query["is_holiday_season"] = data_query["flightDate"].dt.month.isin([6, 7, 12]).astype(int)
data_query["days_to_flight_squared"] = data_query["days_to_flight"] ** 2
data_query["flight_month"] = data_query["flightDate"].dt.month
data_query["flight_year"] = data_query["flightDate"].dt.year
data_query["search_month"] = data_query["searchDate"].dt.month
data_query["search_day"] = data_query["searchDate"].dt.day
data_query["days_to_flight_log"] = np.log1p(data_query["days_to_flight"])

# Remove invalid rows
data_query = data_query[data_query["days_to_flight"] > 0]

# 📌 One-Hot Encode Categorical Features (Airports & Airlines)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = encoder.fit_transform(data_query[['startingAirport', 'destinationAirport', 'segmentsAirlineName']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())

# 📌 Ordinal Encoding for Cabin Class
data_query['segmentsCabinCode'] = data_query['segmentsCabinCode'].fillna('Coach').str.strip().str.title()
data_query = data_query[data_query[ "segmentsCabinCode"].str.strip().str.title() == "Coach"]
data_query.drop(columns=["segmentsCabinCode"], inplace=True)
# 📌 Merge Encoded Data
data_query = data_query.reset_index(drop=True)
data_query = pd.concat([data_query, encoded_df], axis=1)

In [5]:
# :drawing_pin: Prepare Data
# Assuming `data_query` is your dataset and you already have `encoded_df`
X = data_query[['days_to_flight', 'days_to_flight_squared', 'day_of_week', 'is_weekend', 'is_holiday_season',
                'flight_month', 'flight_year', 'search_month', 'search_day', 'days_to_flight_log',
                'seatsRemaining', 'isRefundable'] + list(encoded_df.columns)]
y = data_query['totalFare']
# :drawing_pin: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Combine the data into a DataFrame for PyCaret
data = X_train.copy()
data['totalFare'] = y_train
data = data.dropna(subset=['totalFare'])  # ✅ Drop rows where `totalFare` is NaN


# :drawing_pin: Initialize PyCaret Regression Environment
regression_setup = setup(data, target='totalFare', session_id=123, fold_shuffle=True)
# :drawing_pin: Compare Multiple Models
model_comparison = compare_models()
# :drawing_pin: Display the model comparison table
# This will display a dataframe with the results of all models evaluated
print(model_comparison)
# :drawing_pin: Evaluate the Best Model
best_model = model_comparison
evaluate_model(best_model)
# :drawing_pin: Tune the Best Model (Optional)
tuned_model = tune_model(best_model)
# :drawing_pin: Finalize and Save the Model
final_model = finalize_model(tuned_model)
#save_model(final_model, 'final_regression_model')
# :drawing_pin: Predict on Test Set using the Finalized Model
y_pred_final = predict_model(final_model, data=X_test)
# :drawing_pin: Evaluate the Final Model
mae_final = mean_absolute_error(y_test, y_pred_final['Label'])
mape_final = (mae_final / y_test.mean()) * 100
print(f"\n:bar_chart: Final Model MAE: ${mae_final:.2f}, MAPE: {mape_final:.2f}%")
# :drawing_pin: If you'd like to blend models (like XGBoost and LightGBM), PyCaret can handle ensembling.
# Create an ensemble of the top models (XGBoost, LightGBM, etc.)
blended_model = blend_models([best_model, tuned_model])

,Description,Value
0,Session id,123
1,Target,totalFare
2,Target type,Regression
3,Original data shape,"(69780, 48)"
4,Transformed data shape,"(69780, 48)"
5,Transformed train set shape,"(48846, 48)"
6,Transformed test set shape,"(20934, 48)"
7,Numeric features,46
8,Categorical features,1
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:16:23
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,104.2967,19990.6108,141.3594,0.0778,0.5152,0.5132,0.0800
lr,Linear Regression,104.3147,19992.9997,141.3678,0.0777,0.5152,0.5132,0.3410
ridge,Ridge Regression,104.3144,19992.9743,141.3677,0.0777,0.5152,0.5132,0.1090
lasso,Lasso Regression,104.3171,20006.6998,141.4168,0.0770,0.5156,0.5139,0.8000
llar,Lasso Least Angle Regression,104.3170,20006.6904,141.4167,0.0770,0.5156,0.5139,0.0410
en,Elastic Net,104.5531,20144.8132,141.9056,0.0706,0.5178,0.5171,0.1130
omp,Orthogonal Matching Pursuit,104.9536,20303.4803,142.4644,0.0633,0.5208,0.5205,0.0410
huber,Huber Regressor,101.7411,20760.1261,144.0566,0.0422,0.5037,0.4589,0.2950
knn,K Neighbors Regressor,110.5756,22908.7318,151.3336,-0.0571,0.5422,0.5234,0.2930
par,Passive Aggressive Regressor,128.1805,31467.3316,176.1132,-0.4541,0.6986,0.5331,0.0800


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

: 

In [4]:
# 📌 Select Features
X = data_query[['days_to_flight', 'days_to_flight_squared', 'day_of_week', 'is_weekend', 'is_holiday_season',
                'flight_month', 'flight_year', 'search_month', 'search_day', 'days_to_flight_log',
                'seatsRemaining', 'isRefundable', 'cabin_class_encoded'] + list(encoded_df.columns)]
y = data_query['totalFare']

# 📌 Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 📌 Train XGBoost Model
xgb_model = XGBRegressor(
    n_estimators=500, learning_rate=0.05, max_depth=8,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)
xgb_model.fit(X_train, y_train)

# 📌 Train LightGBM Model
lgbm_model = LGBMRegressor(
    n_estimators=700, learning_rate=0.03, max_depth=10,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)
lgbm_model.fit(X_train, y_train)

# 📌 Make Predictions
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgbm = lgbm_model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052006 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 328
[LightGBM] [Info] Number of data points in the train set: 700000, number of used features: 46
[LightGBM] [Info] Start training from score 265.116839


In [5]:
# 📌 Evaluate Models
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mape_xgb = (mae_xgb / y_test.mean()) * 100

mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
mape_lgbm = (mae_lgbm / y_test.mean()) * 100

print(f"\n📊 XGBoost MAE: ${mae_xgb:.2f}, MAPE: {mape_xgb:.2f}%")
print(f"📊 LightGBM MAE: ${mae_lgbm:.2f}, MAPE: {mape_lgbm:.2f}%")

# 📌 Combine XGBoost and LightGBM Predictions (Weighted Average)
y_pred_blended = (0.7 * y_pred_xgb) + (0.3 * y_pred_lgbm)

# 📌 Evaluate Blended Model
mae_blended = mean_absolute_error(y_test, y_pred_blended)
mape_blended = (mae_blended / y_test.mean()) * 100
print(f"\n📊 Blended Model MAE: ${mae_blended:.2f}, MAPE: {mape_blended:.2f}%")


📊 XGBoost MAE: $54.17, MAPE: 20.44%
📊 LightGBM MAE: $61.12, MAPE: 23.07%

📊 Blended Model MAE: $55.61, MAPE: 20.99%


In [ ]:
# 📌 Find Best Booking Date for Any Flight
flight_date = datetime(2025, 5, 30)
search_dates = [flight_date - timedelta(days=i) for i in range(1, 61)]

# 📌 Create DataFrame for Predictions with All Required Features
search_df = pd.DataFrame({
    'days_to_flight': [(flight_date - d).days for d in search_dates],
    'days_to_flight_squared': [(flight_date - d).days ** 2 for d in search_dates],
    'day_of_week': [d.weekday() for d in search_dates],
    'is_weekend': [1 if d.weekday() >= 5 else 0 for d in search_dates],
    'is_holiday_season': [1 if d.month in [6, 7, 12] else 0 for d in search_dates],
    'flight_month': [d.month for d in search_dates],  # ✅ Added
    'flight_year': [d.year for d in search_dates],  # ✅ Added
    'search_month': [flight_date.month] * len(search_dates),  # ✅ Added
    'search_day': [flight_date.day] * len(search_dates),  # ✅ Added
    'days_to_flight_log': [np.log1p((flight_date - d).days) for d in search_dates],  # ✅ Added

})

# 📌 Add Encoded Features (Default Values)
for col in encoded_df.columns:
    search_df[col] = 0

# 📌 Reorder Columns to Match `X_train`
search_df = search_df[X_train.columns]
# 📌 Predict Prices for Different Booking Dates
predicted_fares = xgb_model.predict(search_df)

# 📌 Find Best Date to Book
best_search_date = search_dates[np.argmin(predicted_fares)]
print(f"\n📅 Best date to book for any flight (May 1st flight): {best_search_date.strftime('%Y-%m-%d')}")

# 📌 Print Total Execution Time
print(f"\n⏳ Total script execution time: {time.time() - start_time:.2f} seconds")


📅 Best date to book for any flight (May 1st flight): 2025-05-24

⏳ Total script execution time: 384.98 seconds
